# Machine learning Wandelaars


## Packages installeren

In [1]:
import pickle
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import colors
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import json

#Kmeans voor preprocessing
from sklearn.linear_model import LinearRegression

# classifiers & testing
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score

## Functie definities

In [2]:
def sliding_window(DataSet):
    """
    Maak van een dataset een patroon via het sliding window principe:
    
    Voor elke array in de data set:
      - bepaal de lengte van de array. Het aantal windows is gelijk aan de lengte van de array -2
        (anders loopt de window over het einde van de array met fout melding als gevolg)
      - vul een window van 3x3 en maak hier een vector van (sla het window plat met hstack).
      - schrijf deze vector weg in de nieuwe dataset
    Schrijf elke nieuwe array weg in de nieuwe dataset


    In: Dataset opgebouwd uit een array van arrays

    Out: Dataset bestaande uit een array met vectoren.
    """

    DataOut=[]
    for InArray in DataSet:
        lengteArray = InArray.shape[0]
        for regelNr in range(lengteArray-2):
            window = (np.hstack((InArray[regelNr],InArray[regelNr+1],InArray[regelNr+2])).tolist())
            DataOut.append(window)
    return np.array(DataOut)

def discretiminator(Dataset,noClusters,Labels):
    """
    Maak van een dataset, aantal clusters en labels een matrix in de vorm van
    lengte dataset x aantal clusters.
    
    Tel vervolgens hoe vaak een aantal clusters per rij voorkomt door 1 op te tellen 
    bij de index die overeenkomt met het cluster (dus als het label de waarde 0 heeft 
    moet er 1 opgeteld worden bij kolom 0. Dus het cluster zelf bepaald bij 
    welke kolom er een wordt opgeteld. Als de lengte van een array uit X_train afgelopen 
    is wordt een volgende rij gemaakt

    In: 
    - Dataset opgebouwd uit een array van arrays. Voor deze case is dit X_train
    - noClusters clusters die in de KMeans is gebruikt en bepaald aantal kolommen van de DataOut
    - Labels bepaald met KMeans

    Out: Dataset bestaande uit een array met vectoren.
    """

    DataOut=np.zeros((len(Dataset),noClusters))
    teller = 0
    for idx,array in enumerate(Dataset):
        for i in range(array.shape[0]-2):
            DataOut[idx,Labels[teller]]+=1
            teller+=1
    return(DataOut)

## Data inlezen
Data inlezen. Trainingsdata in data (X) en klasse/label (y). Data welke voorspelt moet worden in X_pred.

En kijken waar het uit bestaat en hoe het is opgebouwd.

In [3]:
X, y = pickle.load(open("train_X_y.p", "rb"))
X_pred = pickle.load(open('test_X.p', 'rb'))

Lengte?

In [4]:
len(X), len(y), len(X_pred)

(1857, 1857, 626)

Welke klassen/labels zijn er?

In [5]:
np.unique(y)

array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
        11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.])

Hoe vaak komen deze klassen/labels voor?

In [6]:
np.bincount(y)

array([ 63,  48,  15,  90,  13,  60,  46,  43, 100,  39,  70,  60,  83,
       149,  45,  21, 276, 258,  12, 207,  38, 121], dtype=int64)

Wat is het type van de data?

In [7]:
type(X)

list

Hoe zien de elementen van de lijst er uit?

In [8]:
print(type(X[-1]))
print(type(X[-1][0]))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


Dus een lijst met Arrays met Arrays. Opbouw van de eerste array:

In [9]:
X[0]

array([[ -0.081722,   9.9565  ,  -0.23155 ],
       [  1.2667  ,   8.6898  ,  -0.65378 ],
       [  0.38137 ,   9.7249  ,  -0.88532 ],
       [ -0.65378 ,  10.379   ,  -1.076   ],
       [ -0.27241 ,   8.8124  ,   0.23155 ],
       [  0.65378 ,   7.0145  ,   0.92618 ],
       [  0.61292 ,   6.8238  ,   0.19068 ],
       [ -0.14982 ,   7.5048  ,  -0.34051 ],
       [ -0.14982 ,   8.0496  ,  -0.081722],
       [  0.42223 ,   8.1177  ,   0.53119 ],
       [  0.27241 ,   8.3084  ,   0.8036  ],
       [ -0.38137 ,   8.7715  ,   0.95342 ],
       [ -0.46309 ,   9.9973  ,   0.8036  ],
       [  0.040861,  11.795   ,   0.8036  ],
       [ -0.29965 ,  12.531   ,   1.9477  ],
       [ -1.5255  ,  12.871   ,   2.7649  ],
       [ -0.61292 ,  14.288   ,   1.6481  ],
       [  0.92618 ,  12.599   ,   0.14982 ],
       [  2.3427  ,   9.6568  ,  -0.29965 ],
       [  4.1406  ,   8.3084  ,  -0.76274 ],
       [  5.598   ,   8.3084  ,  -0.91256 ],
       [  4.4811  ,   9.003   ,  -0.23155 ],
       [  

Bovenstaande waarde is de lengte van de array die KMeans in moet na uitvoeren van sliding window met een window van 3x3.

### Conclusies over de dataset
Er zijn 22 klasses (0 t/m 21).

Trainingsdata bestaat uit een lijst met arrays met arrays. Deze hebben verschillende lengte, van 45 rijen tot en met 79 rijen, en 3 kolommen.

## Linear regression per array/fragment in dataset

In [13]:

for idx,array in enumerate(X):
    lr=LinearRegression()
    lr.fit(array)

TypeError: fit() missing 1 required positional argument: 'y'